# Data Ingestion — Bronze Layer

Ingest raw CSV files stored in a structured folder hierarchy (`year/month/data.csv`) into a Delta table named **`bronze_data`**, adding ingestion metadata for traceability and enabling incremental data processing.


In [ ]:
from pyspark.sql import functions as f
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
data_path = "dbfs:/mnt/applied_research/landing/*/*/data.csv"

In [ ]:
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(data_path)
)

df_with_ts = df.withColumn("ingestion_timestamp", f.current_timestamp())
table_name = "bronze_data"

In [ ]:
(
    df_with_ts.write
    .format("delta")
    .mode("append")
    .saveAsTable(table_name)
)